# Product Information Gathering System using CrewAI & Hugging Face

This notebook is a comprehensive testing environment for building an autonomous agent system with **CrewAI**. The goal is to collect and analyze information about a specific product by dividing the work among specialized agents.

---

### **Workflow Breakdown:**

1.  **Researcher Agent:** Finds initial product information from online sources.
2.  **Analyst Agent:** Extracts and structures key data points like specifications, features, and reviews.
3.  **Reporter Agent:** Compiles all the structured data into a professional, final report.

---

### **Required Libraries:**

* **crewai:** The core framework for orchestrating the agents.
* **langchain_community:** Provides tools and utilities for the agents.
* **serpapi:** A powerful tool for getting accurate and structured search results.
* **huggingface_hub:** The library to interact with the Hugging Face Inference API.
* **python-dotenv:** To securely manage API keys.

# Environment Setup & Library Imports 

In [1]:
# Import Needed Libraries
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from crewai import LLM
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun 
from langchain.llms import HuggingFaceHub
from langchain.agents import initialize_agent, AgentType
from langchain.agents import tool

In [2]:
# Load environment variables from .env file
load_dotenv()

# Set API keys from environment variables using os.getenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENROUTER_API_KEY")
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
os.environ["OPENAI_MODEL_NAME"] = "openrouter/google/gemini-2.0-flash-exp:free"

# Optional: Print to confirm keys are loaded. Remove this in a final project.
print("API Keys loaded successfully.")

API Keys loaded successfully.


# Define LLM, Tools, and Agents

In [3]:
# Initialize the Hugging Face LLM
llm = ChatOpenAI()

In [4]:
# Define a custom tool for DuckDuckGo search
class MyCustomDuckDuckGoTool(BaseTool):
    name: str = "DuckDuckGo Search Tool"
    description: str = "Search the web for a given query."

    def _run(self, query: str) -> str:
        duckduckgo_tool = DuckDuckGoSearchRun()
        response = duckduckgo_tool.invoke(query)
        return response

In [5]:
# Define the search tool
search_tool = MyCustomDuckDuckGoTool()

In [6]:
# Define the Researcher Agent
researcher = Agent(
    role='Digital Product Scout & Information Hunter',
    goal='Execute deep and targeted web searches to unearth all possible public information on a product, meticulously verifying sources to ensure data accuracy. Your mission is to locate official product pages, comprehensive tech reviews, and authentic user discussions from platforms like Reddit or specialized forums, providing a solid, verifiable foundation for the next stages.',
    backstory=(
        "You are a relentless digital detective. "
        "Your expertise lies in navigating the complex landscape of online information "
        "to find the most relevant and credible data points. "
        "You don't just search; you investigate, cross-referencing information "
        "to weed out misinformation and deliver only the purest, "
        "most reliable data to your team."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[search_tool]
)

In [7]:
# Define the Analyst Agent
analyst = Agent(
    role='Data Synthesis Specialist & Insight Analyst',
    goal='Transform raw, unstructured data from the researcher into a clear, structured, and insightful summary. Your primary objective is to accurately separate technical specifications from marketing claims, identify recurring trends in user sentiment (both positive and negative), and distill complex information into actionable pros and cons that directly address user needs.',
    backstory=(
        "You are the brain of the operation, a methodical expert who sees patterns where others see chaos. "
        "Your talent is in connecting the dots between disparate pieces of information—the technical specs, "
        "the marketing copy, and the genuine user feedback—"
        "to form a coherent, data-driven narrative that reveals the true value and potential pitfalls of a product."
    ),
    verbose=True,
    llm=llm,
    allow_delegation=False,
)

In [8]:
# Define the Reporter Agent
reporter = Agent(
    role='Consumer Product Storyteller & Final Report Editor',
    goal='Synthesize all the analyzed insights into a compelling, easy-to-understand, and professional final report. Your ultimate goal is to translate the technical and analytical findings into a clear, persuasive story for the average consumer, complete with a definitive verdict and a recommendation that is both accurate and trustworthy.',
    backstory=(
        "You are a master of communication, with a knack for turning dry facts into engaging prose. "
        "Your mission is to serve as the voice of the entire operation, "
        "crafting a final document that is not only informative and accurate but also a pleasure to read. "
        "Your work ensures that the team's meticulous research and analysis "
        "reach the end user in the most impactful way possible."
    ),
    verbose=True,
    llm=llm,
    allow_delegation=False,
)

# Define the tasks

In [9]:
# Define the Research Task
research_task = Task(
    description=(
        "Conduct a thorough online search for the product '{product_name}'. "
        "Find its official website, key features, technical specifications, "
        "and user reviews from trusted websites."
    ),
    expected_output=(
        "A detailed summary of the product's features, specifications, and a collection of user reviews."
    ),
    agent=researcher
)

In [10]:
# Define the Analysis Task
analysis_task = Task(
    description=(
        "Analyze the information gathered by the researcher. "
        "Identify the main pros and cons of the product. "
        "Write a comprehensive report that summarizes all findings, "
        "including a final verdict on the product."
    ),
    expected_output=(
        "A comprehensive final report on the product, including a summary of pros, "
        "cons, and an overall verdict."
    ),
    agent=analyst
)

In [11]:
# Define the Reporting Task
reporting_task = Task(
    description=(
        "Take the comprehensive report from the analyst and format it for final presentation. "
        "Ensure the report is well-structured, with clear headings for pros, cons, "
        "and the final verdict. The final output must be a clean, ready-to-publish document."
    ),
    expected_output=(
        "A professionally formatted final report that is ready for publishing."
    ),
    agent=reporter
)

# Gathering the Crew & Running the Process

In [12]:
# Create the crew
product_crew = Crew(
    agents=[researcher, analyst, reporter],
    tasks=[research_task, analysis_task, reporting_task],
    process=Process.sequential,
    verbose=True
)

In [13]:
# Run the crew
product_name = input("Enter the product name you want to research: ")
result = product_crew.kickoff(inputs={'product_name': product_name})

print("Final Report:\n")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d8c40d59-a1e7-4a08-8fce-18a3c5336eec                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Digital Product Scout & Information Hunter                                                              │
│                                                                                                                 │
│  Task: Conduct a thorough online search for the product 'Iphone 15'. Find its official website, key features,   │
│  technical specifications, and user reviews from trusted websites.                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/home/hossam/miniconda3/envs/crewai_env/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Digital Product Scout & Information Hunter                                                              │
│                                                                                                                 │
│  Thought: I need to start by searching for the official website of the iPhone 15, followed by looking for key   │
│  features, technical specifications, and user reviews from trusted websites.                                    │
│                                                                                                                 │
│  Using Tool: DuckDuckGo Search Tool                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"iPhone 15 official website\"}"                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Consumer Product Storyteller & Final Report Editor                                                      │
│                                                                                                                 │
│  Task: Take the comprehensive report from the analyst and format it for final presentation. Ensure the report   │
│  is well-structured, with clear headings for pros, cons, and the final verdict. The final output must be a      │
│  clean, ready-to-publish document.                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Consumer Product Storyteller & Final Report Editor                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  iPhone 15 - Comprehensive Review                                                                               │
│                                                                                                                 │
│  Overview:                                                                                                      │
│  Introducing the iPhone 15, Apple's latest flagship smartphone that pushes the boundaries of technology and     │
│  elevates the user experience. Crafted with precision and innovation, this device aims to redefine the          │
│  smartphone landscape with its cutting-edge features and seamless performance.                                  │
│                                                                                                                 │
│  Key Features:                                                                                                  │
│                                                                                                                 │
│  1. Display:                                                                                                    │
│  Experience brilliance like never before with the iPhone 15's breathtaking 6.7-inch OLED display. Boasting      │
│  True Tone technology, this display delivers vibrant colors and deep blacks, creating an immersive visual       │
│  feast for your eyes.                                                                                           │
│                                                                                                                 │
│  2. Performance:                                                                                                │
│  Powered by the mighty A16 Bionic chip, the iPhone 15 is a powerhouse of performance. Say goodbye to lags and   │
│  delays as this device offers lightning-fast speeds and seamless multitasking capabilities, redefining          │
│  efficiency on the go.                                                                                          │
│                                                                                                                 │
│  3. Camera:                                                                                                     │
│  Unleash your inner photographer with the iPhone 15's advanced camera system. Featuring a triple-lens setup     │
│  with improved low-light performance and AI-powered features, capturing stunning photos and videos has never    │
│  been easier.                                                                                                   │
│                                                                                                                 │
│  4. Connectivity:                                                                                               │
│  Stay connected at the speed of light with 5G support on the iPhone 15. Whether you're downloading large files  │
│  or streaming content, this device ensures a seamless and uninterrupted online experience wherever you are.     │
│                                                                                                                 │
│  5. Battery Life:                                                                                               │
│  Bid farewell to battery anxiety with the iPhone 15's o

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: fd7d8014-9997-423e-9b5d-bac7e63ef11f                                                                     │
│  Agent: Consumer Product Storyteller & Final Report Editor                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d8c40d59-a1e7-4a08-8fce-18a3c5336eec                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: iPhone 15 - Comprehensive Review                                                                 │
│                                                                                                                 │
│  Overview:                                                                                                      │
│  Introducing the iPhone 15, Apple's latest flagship smartphone that pushes the boundaries of technology and     │
│  elevates the user experience. Crafted with precision and innovation, this device aims to redefine the          │
│  smartphone landscape with its cutting-edge features and seamless performance.                                  │
│                                                                                                                 │
│  Key Features:                                                                                                  │
│                                                                                                                 │
│  1. Display:                                                                                                    │
│  Experience brilliance like never before with the iPhone 15's breathtaking 6.7-inch OLED display. Boasting      │
│  True Tone technology, this display delivers vibrant colors and deep blacks, creating an immersive visual       │
│  feast for your eyes.                                                                                           │
│                                                                                                                 │
│  2. Performance:                                                                                                │
│  Powered by the mighty A16 Bionic chip, the iPhone 15 is a powerhouse of performance. Say goodbye to lags and   │
│  delays as this device offers lightning-fast speeds and seamless multitasking capabilities, redefining          │
│  efficiency on the go.                                                                                          │
│                                                                                                                 │
│  3. Camera:                                                                                                     │
│  Unleash your inner photographer with the iPhone 15's advanced camera system. Featuring a triple-lens setup     │
│  with improved low-light performance and AI-powered features, capturing stunning photos and videos has never    │
│  been easier.                                                                                                   │
│                                                                                                                 │
│  4. Connectivity:                                                                                               │
│  Stay connected at the speed of light with 5G support on the iPhone 15. Whether you're downloading large files  │
│  or streaming content, this device ensures a seamless and uninterrupted online experience wherever you are.     │
│                                                                                                                 │
│  5. Battery Life:                                     

Final Report:

iPhone 15 - Comprehensive Review

Overview:
Introducing the iPhone 15, Apple's latest flagship smartphone that pushes the boundaries of technology and elevates the user experience. Crafted with precision and innovation, this device aims to redefine the smartphone landscape with its cutting-edge features and seamless performance.

Key Features:

1. Display:
Experience brilliance like never before with the iPhone 15's breathtaking 6.7-inch OLED display. Boasting True Tone technology, this display delivers vibrant colors and deep blacks, creating an immersive visual feast for your eyes.

2. Performance:
Powered by the mighty A16 Bionic chip, the iPhone 15 is a powerhouse of performance. Say goodbye to lags and delays as this device offers lightning-fast speeds and seamless multitasking capabilities, redefining efficiency on the go.

3. Camera:
Unleash your inner photographer with the iPhone 15's advanced camera system. Featuring a triple-lens setup with improved low-light p